In [1]:
import ipywidgets.widgets as widgets
from time import sleep, time
import sys
sys.path.append('/home/ballsbot/projects/ballsbot/python/lib')
from ballsbot.camera import get_images_and_cameras
from ballsbot.servos import get_controls
from ballsbot.utils import keep_rps
from ballsbot.lidar import calibration_to_xywh
from ballsbot.lidar_with_memory import LidarWithMemory
from ballsbot.utils import run_as_thread
from ballsbot.tracking import Tracker
from ballsbot.controller import link_controller
from ballsbot.joystick import JoystickWrapper
from ballsbot_localization import ballsbot_localization as grid
from ballsbot.drawing import BotPoseAbsCoordsDrawing
from ballsbot.dashboard import Dashboard
from ballsbot.ros_messages import get_ros_messages

In [2]:
get_ros_messages().start(sync=False)

In [3]:
lidar = LidarWithMemory()
tracker = Tracker(lidar)
tracker.start()

In [4]:
controller = widgets.Controller(index=0)  # http://html5gamepad.com
display(controller)

images = []
for image, camera in get_images_and_cameras():
    images.append(image)
hbox = widgets.HBox(images)
display(hbox)

dashboard = Dashboard(widgets)

lidar_drawing_raw = BotPoseAbsCoordsDrawing(dashboard, 'lidar')
dashboard.add_subplot('lidar', lidar_drawing_raw.get_drawing_func_name())

display(dashboard.get_image())

Controller()

Image(value=b'', format='jpeg', height='640', width='640')

In [5]:
link_controller(JoystickWrapper(controller))

In [6]:
def update_images():
    ts = None
    while True:
        ts = keep_rps(ts, fps=1)

        params = tracker.get_picture_params(with_free_tiles=True)
        if params:
            lidar_drawing_raw.update_image(*params, only_nearby_meters=6.)
            dashboard.redraw()

run_as_thread(update_images)

In [8]:
# from ballsbot.utils import join_all_threads
# join_all_threads()